In [135]:
import requests, json, re
import matplotlib.pyplot as plt
from datetime import date

In [136]:
def api(method, parameters):
    req = 'https://api.vk.com/method/' + method + '?'
    for k in parameters:
        req += k + '=' + parameters[k] + '&'
    return json.loads(requests.get(req[:-1]).text)

In [137]:
def writefile(name, text):
    f = open(name, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [158]:
def get_comments(post_id, num_com, lencoms, cities, ages):
    allcoms = ''
    averlencoms = []
    comments = api('wall.getComments', {'owner_id':'-91933860', 'post_id': str(post_id), 'count':'100'})
    if num_com > 100:
        comments['response'] += api('wall.getComments', {'owner_id':'-91933860', 'post_id': post_id, 'count':'100', 'offset':'100'})['response']
    for com in comments['response']:
        if type(com) == dict:
            text = com['text'].replace('<br>', '\n')
            allcoms += '* ' + cleaning(text) + '\n'
            if com['from_id'] > 0:
                cities, ages = get_info(com['from_id'], cities, ages)
                lencoms.append(len(text.split()))
                averlencoms.append(len(text.split()))
    #writefile(str(post_id) + ' comments.txt', allcoms)
    if len(averlencoms) != 0:
        average = sum(averlencoms)/len(averlencoms)
    else:
        average = 0
    return lencoms, cities, ages, average     

In [139]:
def get_age(bdate):
    bdate = bdate.split()
    if len(bdate) < 3:
        return 'unknown'
    today = date.today()
    age = today.year - bdate[2]
    if today.month < bdate[1]:
        age -= 1
    elif today.month == bdate[1] and today.day < bdate[0]:
        age -= 1
    return age

In [140]:
def get_info(author, cities, ages):
    info = api('users.get', {'user_ids':str(author), 'fields':'city,bdate'})
    if 'city' in info['response'][0]:    
        cities.append(info['response'][0]['city'])
    else:
        cities += '0'
    if 'bdate' in info['response'][0]:    
        ages += get_age(info['response'][0]['bdate'])
    else:
        ages += 'unknown'
    return cities, ages

In [141]:
def cleaning(text):
    import sys
    non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
    text = re.sub('\[id[0-9]*\|', '', text)
    text = re.sub('\]', '', text)
    return text.translate(non_bmp_map)

In [162]:
def newplot(x, y, x_label, y_label, name):
    plt.plot(x, y)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.savefig(name +'.png', format='png', dpi=100)
    plt.clf()

In [143]:
def age_lencoms(ages, lencoms):
    dict_ages = {}
    for i in range(len(ages)):
        if ages[i] != 'unknown':
            if ages[i] not in dict_ages:
                dict_ages[ages[i]] = [lencoms[i]]
            else:
                dict_ages[ages[i]].append(lencoms[i])
    return dict_ages
        

In [144]:
def city_lencoms(cities, lencoms):
    dict_cities = {}
    for i in range(len(cities)):
        if cities[i] != '0' and cities[i] != 0:
            if cities[i] not in dict_cities:
                dict_cities[cities[i]] = [lencoms[i]]
            else:
                dict_cities[cities[i]].append(lencoms[i])
    return dict_cities

In [160]:
def main():
    posts = api('wall.get', {'owner_id':'-91933860', 'count':'100'})
    posts['response'] += api('wall.get', {'owner_id':'-91933860', 'count':'100', 'offset':'100'})['response']
    cities, lencoms, lenposts, ages, averlencoms = [], [], [], [], []
    for post in posts['response']:
        if type(post) == dict:
            post_id = post['id']
            text = post['text'].replace('<br>', '\n')
            #writefile(str(post_id) + '.txt', text)
            num_com = post['comments']['count']
            lencoms, cities, ages, average = get_comments(post_id, num_com, lencoms, cities, ages)
            lenposts.append(len(text.split()))
            averlencoms.append(average)
    newplot(lenposts, averlencoms, 'Длина поста', 'Средняя длина комментария', 'lenlen')
    dict_ages = age_lencoms(ages,lencoms)
    dict_cities = city_lencoms(cityes,lencoms)
    
    newplot(dict_ages.keys(), [dict_ages[key](sum)/len(dict_ages[key]) for key in dict_ages], 'возраст', 'средняя длина комментария', 'age')
    newplot(dict_citiees.keys(), [dict_cities[key](sum)/len(dict_cities[key]) for key in dict_cities], 'город', 'средняя длина комментария', 'city')

In [ ]:
if __name__ == '__main__':
    main()